In [ ]:
from datetime import datetime, date
from pathlib import PurePosixPath
from pyspark.sql import SparkSession

# Configuração
CATALOG = "workspace"
SCHEMA  = "default"
VOLUME  = "elt_volume"

VOL_ROOT = PurePosixPath("/Volumes") / CATALOG / SCHEMA / VOLUME
GOLD_ROOT = VOL_ROOT / "gold"
today_str = date.today().isoformat()
time_str  = datetime.now().strftime("%H%M")  # Current time: 1503
GOLD_PART = GOLD_ROOT / today_str.replace("-", "/")
GOLD_BASE = GOLD_PART / f"vendas_gold_final_{time_str}"

spark = SparkSession.builder.getOrCreate()

# Recupera caminho do silver_csv da task anterior
task_key = "Execute_process_silver_clean"  # Nome real da task no Workflow
try:
    SILVER_CSV = dbutils.jobs.taskValues.get(taskKey=task_key, key="silver_csv", debugValue="Not set")
    print(f"DEBUG: Retrieved silver_csv from task values: {SILVER_CSV}")
    if not SILVER_CSV or not dbutils.fs.ls(str(SILVER_CSV)):
        raise FileNotFoundError(f"Arquivo não encontrado em {SILVER_CSV}")
except Exception as e:
    print(f"DEBUG: Erro ao recuperar silver_csv: {str(e)}")
    # Fallback: Procurar o diretório mais recente criado pelo Silver
    SILVER_ROOT = VOL_ROOT / "silver"
    silver_base = str(SILVER_ROOT / today_str.replace("-", "/"))
    silver_files = dbutils.fs.ls(silver_base)
    SILVER_CSV = None
    if silver_files:
        # Ordena por tempo de modificação para pegar o diretório mais recente
        silver_dirs = sorted([f for f in silver_files if f.isDir() and f.name.startswith("vendas_silver_processed_")], key=lambda x: x.modificationTime, reverse=True)
        for sub_dir_entry in silver_dirs:
            sub_dir = sub_dir_entry.path.replace("dbfs:", "")
            sub_files = dbutils.fs.ls(sub_dir)
            for sf in sub_files:
                if sf.name.endswith(".csv"):
                    SILVER_CSV = sf.path.replace("dbfs:", "")
                    print(f"DEBUG: Found candidate file in {sub_dir}: {SILVER_CSV}")
                    break
            if SILVER_CSV:
                break
    if not SILVER_CSV or not dbutils.fs.ls(str(SILVER_CSV)):
        raise FileNotFoundError(f"Arquivo não encontrado em subdiretórios de {silver_base}")
    print(f"DEBUG: Fallback to constructed path: {SILVER_CSV}")

# Cria diretórios necessários
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")
dbutils.fs.mkdirs(str(GOLD_PART))
print(f"DEBUG: Criado diretório {GOLD_PART}")

# Leitura do CSV da camada silver
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(f"dbfs:{SILVER_CSV}"))
print(f"DEBUG: Arquivo CSV lido de {SILVER_CSV}, total de linhas: {df.count()}")

# Seleção dos campos essenciais para a camada gold
columns_to_keep = [
    "venda_id",
    "data",
    "vendedor_id",
    "produto_id",
    "categoria",
    "valor_liquido",
    "desconto",
    "score_venda",
    "flag_anomalia"
]
df_gold = df.select(columns_to_keep)

# Contagem de linhas processadas
total_rows_processed = df_gold.count()
print(f"DEBUG: Total de linhas processadas: {total_rows_processed}")

# Salva o arquivo final em formato CSV
df_gold.coalesce(1).write.mode("overwrite").option("header", "true").option("encoding", "UTF-8").csv(f"dbfs:{GOLD_BASE}")
print(f"DEBUG: Arquivo salvo em {GOLD_BASE}")

# Encontra o primeiro arquivo particionado para task value
gold_files = dbutils.fs.ls(f"dbfs:{GOLD_BASE}")
if not gold_files:
    raise FileNotFoundError(f"Nenhum arquivo encontrado em {GOLD_BASE}")
gold_csv_path = [f.path.replace("dbfs:", "") for f in gold_files if f.path.endswith(".csv")][0]
print(f"DEBUG: Primeiro arquivo particionado encontrado: {gold_csv_path}")

# Registro de metadados
metadata = {
    "total_rows_processed": total_rows_processed,
    "output_path": gold_csv_path
}

# Salva metadados como JSON
metadata_path = GOLD_PART / f"metadata_vendas_gold_{time_str}.json"
dbutils.fs.put(f"dbfs:{metadata_path}", str(metadata), overwrite=True)
print(f"DEBUG: Metadados salvos em {metadata_path}")

# Salva contextos para próximas tasks
try:
    dbutils.jobs.taskValues.set(key="gold_csv", value=gold_csv_path)
    dbutils.jobs.taskValues.set(key="gold_metadata", value=str(metadata_path))
    print(f"DEBUG: Task values successfully set - gold_csv: {gold_csv_path}")
except Exception as e:
    print(f"DEBUG: Failed to set task values: {str(e)}")
    raise

print(f"DEBUG: Set gold_csv: {gold_csv_path}")
print(f"DEBUG: Set gold_metadata: {str(metadata_path)}")